# 作業目標:
    
    建立一個網路模型
    
# 作業重點:

請修改 Name 中, 自定義的 Layer 名稱

增加一層全連階層

宣告 MODEL API, 分別採用自行定義的 Input/Output Layer

model.summary 查看 Layers stack

In [26]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# 接收輸入的新聞標題，即表示一個整數序列(每個詞都有整數編碼)
# 這些整數介於 1-10000 之間(因為有 10000 個詞彙表)，且序列長度為 100 個詞
# 宣告一個 NAME 去定義 Input
main_input = Input(shape=(100,), dtype='int32', name='main_input')


# Embedding Layer 將輸入序列做編碼，成為一個稠密向量的序列
# 每一個向量的維度為 512
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# LSTM Layer 將向量序列轉換成為單個向量
# 它包含了整個序列的上下文訊息
lstm_out = LSTM(32)(x)

In [27]:
# 插入輔助損失計算，使得在主要模型中主損失很高的情況下
# LSTM Layer 和 Embedding Layer 都能被平穩地訓練
news_output = Dense(1, activation='sigmoid', name='news_out')(lstm_out)

In [28]:
# 輔助輸入數據與 LSTM Layer 的輸出連接，輸入到模型
import keras
news_input = Input(shape=(5,), name='news_in')

x = keras.layers.concatenate([lstm_out, news_input])


# 堆疊多個全連接網路層
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)


# 最後添加主要的邏輯迴歸層
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

**作業：宣告 MODEL API, 分別採用自行定義的 Input/Output Layer**

In [29]:
# 宣告 MODEL API, 分別採用自行定義的 Input/Output Layer
model = Model(inputs=[main_input, news_input], outputs=[main_output, news_output])

**透過 get_layer() 的方法來觀察 layer 的資訊**

In [30]:
print(model.get_layer('main_input'))
print(model.get_layer('news_in'))

In [31]:
print(model.get_layer('main_output'))
print(model.get_layer('news_out'))

**透過 compile( ) 來定義損失函數(loss)、優化函數(optimizer)以及損失權重等等**

In [33]:
model.compile(optimizer='rmsprop',
              loss={'main_output':'binary_crossentropy', 'news_out':'binary_crossentropy'},
              loss_weights={'main_output':1., 'news_out':0.2})

In [35]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 512)     5120000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 32)           69760       embedding_2[0][0]                
__________________________________________________________________________________________________
news_in (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
concatenat